# 프롬프트 엔지니어링과 Tool Use를 활용한 미국 주식 분석 서비스 (Basic)
이 코드는 Alpha Vantage API를 활용하여 사용자가 입력한 주식 종목(ticker)의 주요 데이터를 가져옵니다. 구체적으로 전일 종가, 변동률, 당일 거래 가격 범위(최고가/최저가), 시가, 그리고 거래량 정보를 수집합니다. 이렇게 얻은 데이터는 Claude 3 Sonnet 모델에 전달되어 분석되며, 그 결과를 자연어 형태로 사용자에게 제공합니다.

## 1. 환경 설정

In [1]:
!python -V

Python 3.10.14


In [2]:
# 사용하기 전에 필요한 설치:
%pip install pandas numpy requests lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = "../.."
add_python_path(module_path)

python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/13_tool_use_stock_analysis', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr']


Setup AWS Bedrock and Claude Integration

In [4]:
from pprint import pprint
from termcolor import colored
from utils import bedrock
from utils.bedrock import *
from anthropic import AnthropicBedrock

# Initialize Bedrock client
client = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))


Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-5-V-2-Sonnet': 'anthropic.claude-3-5-sonnet-20241022-v2:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.l

## 2. Tool 정의

Alpha Vantage를 사용해 최신 주식 데이터 가져오기
- [사전 사항] Alpha Vantage API 발급 받기 https://www.alphavantage.co/support/#api-key
- [참고] Alpha Vantage의 문서에 따르면, 프리티어에서 기본적으로 데이터는 "각 거래일 종료 시점에 업데이트" 됨

In [5]:
import requests

def get_stock_data(ticker):
    """Alpha Vantage에서 주식 데이터를 가져오는 함수"""
    try:
        api_key = "EL41YYH6U3LRDXFC"  # Alpha Vantage API 키
        url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={ticker}&apikey={api_key}"
        
        response = requests.get(url)
        data = response.json()
        
        if "Global Quote" not in data:
            return {
                'status': 'error',
                'message': 'No data available'
            }
            
        quote = data["Global Quote"]
        
        return {
            'status': 'success',
            'data': {
                'date': quote["07. latest trading day"],
                'open': float(quote["02. open"]),
                'high': float(quote["03. high"]),
                'low': float(quote["04. low"]),
                'volume': int(quote["06. volume"])
            }
        }
    except Exception as e:
        return {
            'status': 'error',
            'message': str(e)
        }

## 3. Claude 호출

(참고) Tool Use 패턴의 일반적인 흐름
1. 도구 사용 필요성 판단 및 요청
2. 도구 실행 결과를 바탕으로 최종 응답 생성

In [6]:
def process_stock_request(client, ticker):
    """Bedrock Converse API를 사용하여 주식 데이터 처리"""
    try:
        # Tool 설정
        tool_config = {
            'tools': [{
                'toolSpec': {
                    # Tool의 이름, 설명, 입력 파라미터 정의
                    'name': 'get_stock_data',
                    'description': 'Fetches current stock data from Alpha Vantage API',
                    'inputSchema': {
                        'json': {
                            'type': 'object',
                            'properties': {
                                'ticker': {
                                    'type': 'string',
                                    'description': 'Stock ticker symbol'
                                }
                            },
                            'required': ['ticker']
                        }
                    }
                }
            }],
            'toolChoice': {'auto': {}} # 자동 Tool 선택 설정 (any/auto/tool 중에 한개 입력 가능)
        }

        # System prompt
        # AI의 역할과 응답 형식 정의
        # 숫자 포맷팅 규칙 지정
        # 날짜 정보 포함 요구사항 명시
        system_prompt = [{
         "text": """You are a financial data analyst assistant. When given a stock ticker, 
            use the get_stock_data tool to fetch and analyze trading data. 
            Format your response as follows:
            1. Trading Date
            2. Current Price and Change Percentage
            3. Day's Trading Range (High/Low)
            4. Opening Price
            5. Trading Volume (use comma formatting)
            
            Make the response clear and easy to read. Format numbers with appropriate decimal places 
            and use commas for thousands in volume figures."""
        }]

        # 메시지 구성
        messages = [{
            'role': 'user',
            'content': [{'text': f'Get me the current trading data for {ticker}'}]
        }]

        # 첫번째 Converse API 호출
        # 목적: 사용자의 초기 요청을 처리하고 필요한 도구(tool) 사용을 결정
        # 모델이 주식 데이터를 가져오기 위해 tool을 사용해야 한다고 판단하면 stopReason='tool_use'를 반환
        response = client.converse(
            modelId='anthropic.claude-3-sonnet-20240229-v1:0',  # 적절한 모델 ID로 변경
            messages=messages,
            toolConfig=tool_config,
            inferenceConfig={
                'temperature': 0.5, # temperature 0.5로 설정하여 일관된 응답 생성
                'maxTokens': 500
            }
        )

        # Tool 호출이 필요한 경우
        if response['stopReason'] == 'tool_use':
            tool_use = response['output']['message']['content'][0]['toolUse']
            print(tool_use)
            stock_data = get_stock_data(ticker)
            print(stock_data)
            
            # Tool 실행 결과를 대화 컨텍스트에 추가 (메시지 체인 구성)
            messages.append({
                'role': 'assistant',
                'content': [{
                    'toolUse': tool_use
                }]
            })
            messages.append({
                'role': 'user',
                'content': [{
                    'toolResult': {
                        'toolUseId': tool_use['toolUseId'],
                        'content': [{'json': stock_data}],
                        'status': 'success'
                    }
                }]
            })

            # 두번째 Converse API 호출 
            # 목적: tool 실행 결과를 받아서 최종 응답을 생성
            # messages 배열에 tool 실행 결과가 포함되어 있음
            # 모델이 tool 실행 결과를 해석하고 사용자가 이해하기 쉬운 형태로 응답을 생성
            final_response = client.converse(
                modelId='anthropic.claude-3-sonnet-20240229-v1:0',
                system=system_prompt,
                messages=messages,
                toolConfig=tool_config,
                inferenceConfig={
                    "temperature": 0.7, # temperature 0.7로 설정하여 더 자연스러운 응답 생성
                    "maxTokens": 500,
                    "stopSequences": []
                }
            )
            
            return final_response['output']['message']['content'][0]['text']
        
        return response['output']['message']['content'][0]['text']

    except Exception as e:
        return f"Error processing request: {str(e)}"

In [8]:
ticker = input("Enter stock ticker (or 'quit' to exit): ").upper()
# ticker = 'AMZN'
result = process_stock_request(client, ticker)
print("\nResult:")
print(result)
print("-" * 50)

{'toolUseId': 'tooluse_fxNF8WAUT7y43fnCwz0dZg', 'name': 'get_stock_data', 'input': {'ticker': 'AMZN'}}
{'status': 'success', 'data': {'date': '2024-11-12', 'open': 208.37, 'high': 209.54, 'low': 206.01, 'volume': 38942918}}

Result:
1. Trading Date: 2024-11-12

2. Current Price: $207.73 (-0.31%)

3. Day's Trading Range: $206.01 / $209.54

4. Opening Price: $208.37

5. Trading Volume: 38,942,918

The data shows that on November 12, 2024, Amazon (AMZN) opened trading at $208.37. The stock price fluctuated between a low of $206.01 and a high of $209.54 throughout the day. By market close, the price settled at $207.73, down 0.31% from the previous day's closing price. The trading volume for the day was a substantial 38,942,918 shares exchanged.
--------------------------------------------------
